使用資料增強技術搭配正則化(regularization)技術，例如: L1, L2

In [11]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Version A：原始資料（無增強）
raw_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225])
])

# Version B：訓練用資料增強
aug_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomRotation(5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.3,
                             scale=(0.02, 0.15),
                             ratio=(0.3, 3.3),
                             value=0)  # 或用 'random'
])

# 載入訓練／測試 Dataset
raw_train_full = datasets.ImageFolder(root="./data/train", transform=raw_transform)
raw_test     = datasets.ImageFolder(root="./data/test",  transform=raw_transform)

aug_train_full = datasets.ImageFolder(root="./data/train", transform=aug_transform)
#raw_transform 以評估增強後模型真實表現
aug_val_full  = datasets.ImageFolder(root="./data/train", transform=raw_transform)
aug_test      = datasets.ImageFolder(root="./data/test",  transform=raw_transform)

print(" 類別對應：", raw_train_full.classes)
print(" 訓練筆數（原始）：", len(raw_train_full))
print(" 測試筆數：", len(raw_test))

# 切分訓練 / 驗證集（8:2）
train_ratio = 0.8
raw_train_size = int(len(raw_train_full) * train_ratio)
raw_val_size   = len(raw_train_full) - raw_train_size
raw_train, raw_val = random_split(raw_train_full, [raw_train_size, raw_val_size])

aug_train_size = int(len(aug_train_full) * train_ratio)
aug_val_size   = len(aug_train_full) - aug_train_size
aug_train, _   = random_split(aug_train_full, [aug_train_size, len(aug_train_full)-aug_train_size])
_, aug_val     = random_split(aug_val_full, [aug_train_size, len(aug_val_full)-aug_train_size])

# DataLoader
batch_size = 64
raw_train_loader = DataLoader(raw_train, batch_size=batch_size, shuffle=True)
raw_val_loader   = DataLoader(raw_val,   batch_size=batch_size, shuffle=False)
raw_test_loader  = DataLoader(raw_test,  batch_size=batch_size, shuffle=False)

aug_train_loader = DataLoader(aug_train, batch_size=batch_size, shuffle=True)
aug_val_loader   = DataLoader(aug_val,   batch_size=batch_size, shuffle=False)
aug_test_loader  = DataLoader(aug_test,  batch_size=batch_size, shuffle=False)


 類別對應： ['men', 'women']
 訓練筆數（原始）： 220
 測試筆數： 80


In [ ]:
import copy
import torch
import torch.nn as nn

def train_model(model,train_loader,val_loader,num_epochs,device,l1_lambda=1e-5,label="", ):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-3, weight_decay=1e-2)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

    best_acc = 0.0
    best_state = copy.deepcopy(model.state_dict())

    print(f"開始訓練 (L1 λ={l1_lambda}): {label}")
    for epoch in range(num_epochs):
        # --- 訓練階段 ---
        model.train()
        running_loss = 0.0
        correct, total = 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            # model.forward 回傳 (outputs, l1_norm)
            outputs, l1_norm = model(images)
            loss = criterion(outputs, labels)
            if l1_lambda > 0:
                loss = loss + l1_lambda * l1_norm

            loss.backward()
            optimizer.step()
            scheduler.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total

        # --- 驗證階段 ---
        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs, _ = model(images)  # 忽略 l1_norm
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)
        val_acc = 100 * val_correct / val_total

        print(
            f"[{label}] Epoch {epoch+1}/{num_epochs}  "
            f"Loss: {running_loss:.4f}  "
            f"Train Acc: {train_acc:.2f}%  Val Acc: {val_acc:.2f}%"
        )

        if val_acc > best_acc:
            best_acc = val_acc
            best_state = copy.deepcopy(model.state_dict())

    # 載入最佳權重
    model.load_state_dict(best_state)
    print(f"[{label}] 最佳驗證準確率 (L1 λ={l1_lambda}): {best_acc:.2f}%")
    return model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
def evaluate_model(model, test_loader, device, label=""):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            # 拆解 forward 回傳值
            outputs, _ = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"[{label}] 測試集準確率：{acc:.2f}%")
    return acc


L1

In [ ]:
import torchvision.models as models
import torch.nn as nn

class GenderCNN_L1(nn.Module):
    def __init__(self):
        super().__init__()
        # 載入預訓練 EfficientNet‑B0
        self.backbone = models.efficientnet_b0(pretrained=True)
        # 冻结大部分層，只解凍最後五個 block
        for param in self.backbone.parameters():
            param.requires_grad = False
        for param in self.backbone.features[-5:].parameters():
            param.requires_grad = True

        
        self.backbone.classifier[1] = nn.Sequential(
            nn.Linear(1280, 512),
            nn.ReLU(inplace=True),

            nn.Linear(512, 256),
            nn.ReLU(inplace=True),

            nn.Linear(256, 2)
        )

    def forward(self, x):
        outputs = self.backbone(x)
        # 計算 L1 範數
        l1_norm = sum(p.abs().sum() for p in self.parameters() if p.requires_grad)
        return outputs, l1_norm


In [15]:
# 參數
num_epochs = 20
l1_lambda  = 1e-5
# 訓練 raw
model_raw = GenderCNN_L1().to(device)
model_raw = train_model(model_raw, raw_train_loader, raw_val_loader, num_epochs, device, l1_lambda=l1_lambda, label="raw")

# 訓練 augmented
model_aug = GenderCNN_L1().to(device)
model_aug = train_model( model_aug, aug_train_loader, aug_val_loader, num_epochs, device,l1_lambda=l1_lambda, label="augmented"
)


開始訓練 (L1 λ=1e-05): raw
[raw] Epoch 1/20  Loss: 10.2883  Train Acc: 60.80%  Val Acc: 50.00%
[raw] Epoch 2/20  Loss: 9.4840  Train Acc: 80.68%  Val Acc: 79.55%
[raw] Epoch 3/20  Loss: 8.4376  Train Acc: 98.30%  Val Acc: 86.36%
[raw] Epoch 4/20  Loss: 8.1746  Train Acc: 100.00%  Val Acc: 90.91%
[raw] Epoch 5/20  Loss: 8.1423  Train Acc: 100.00%  Val Acc: 88.64%
[raw] Epoch 6/20  Loss: 8.1321  Train Acc: 100.00%  Val Acc: 88.64%
[raw] Epoch 7/20  Loss: 8.1340  Train Acc: 99.43%  Val Acc: 81.82%
[raw] Epoch 8/20  Loss: 8.1274  Train Acc: 99.43%  Val Acc: 84.09%
[raw] Epoch 9/20  Loss: 8.1111  Train Acc: 100.00%  Val Acc: 86.36%
[raw] Epoch 10/20  Loss: 8.1128  Train Acc: 100.00%  Val Acc: 86.36%
[raw] Epoch 11/20  Loss: 8.1063  Train Acc: 100.00%  Val Acc: 86.36%
[raw] Epoch 12/20  Loss: 8.2207  Train Acc: 98.30%  Val Acc: 86.36%
[raw] Epoch 13/20  Loss: 8.1120  Train Acc: 100.00%  Val Acc: 86.36%
[raw] Epoch 14/20  Loss: 8.1040  Train Acc: 100.00%  Val Acc: 86.36%
[raw] Epoch 15/20  Loss: 

In [18]:
evaluate_model(model_raw, raw_test_loader, device, label="raw")
evaluate_model(model_aug, aug_test_loader, device, label="augmented")

[raw] 測試集準確率：86.25%
[augmented] 測試集準確率：90.00%


90.0

L2

In [19]:
import copy
import torch
import torch.nn as nn

def train_model_l2(model,train_loader, val_loader, num_epochs, device, reg_lambda=1e-4, reg_type='l2', label=''):
 
    criterion = nn.CrossEntropyLoss()
    # weight_decay = 0 because we apply explicit reg in the loss
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=0)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

    best_acc = 0.0
    best_state = copy.deepcopy(model.state_dict())

    print(f"開始訓練 ({reg_type.upper()} λ={reg_lambda}): {label}")
    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            # model.forward 必須回傳 (outputs, reg_norm)
            outputs, reg_norm = model(images)
            loss = criterion(outputs, labels) + reg_lambda * reg_norm

            loss.backward()
            optimizer.step()
            scheduler.step()

            running_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total

        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs, _ = model(images)  # 忽略 reg_norm
                preds = outputs.argmax(dim=1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = 100 * val_correct / val_total
        print(
            f"[{label}] Epoch {epoch}/{num_epochs}  "
            f"Loss: {running_loss:.4f}  "
            f"Train Acc: {train_acc:.2f}%  "
            f"Val Acc: {val_acc:.2f}%"
        )

        if val_acc > best_acc:
            best_acc = val_acc
            best_state = copy.deepcopy(model.state_dict())

    model.load_state_dict(best_state)
    print(f"[{label}] 最佳驗證準確率 ({reg_type.upper()} λ={reg_lambda}): {best_acc:.2f}%")
    return model


In [20]:
import torchvision.models as models
import torch.nn as nn

class GenderCNN_L2(nn.Module):
    def __init__(self):
        super().__init__()
        # 載入預訓練 EfficientNet‑B0
        self.backbone = models.efficientnet_b0(pretrained=True)
    
        for param in self.backbone.parameters():
            param.requires_grad = False
        for param in self.backbone.features[-5:].parameters():
            param.requires_grad = True

        
        self.backbone.classifier[1] = nn.Sequential(
            nn.Linear(1280, 512),
            nn.ReLU(inplace=True),

            nn.Linear(512, 256),
            nn.ReLU(inplace=True),

            nn.Linear(256, 2)
        )

    def forward(self, x):
        outputs = self.backbone(x)
        # 計算 L1 範數
        l2_norm = sum(p.pow(2).sum() for p in self.parameters() if p.requires_grad)
        return outputs, l2_norm


In [21]:
num_epochs = 20
l2_lambda  = 1e-4

model_raw = GenderCNN_L2().to(device)
model_raw = train_model_l2(model_raw, raw_train_loader, raw_val_loader, num_epochs, device, l2_lambda, label="raw")

model_aug = GenderCNN_L2().to(device)
model_aug = train_model_l2(model_aug, aug_train_loader, aug_val_loader, num_epochs, device, l2_lambda, label="augmented")


開始訓練 (L2 λ=0.0001): raw
[raw] Epoch 1/20  Loss: 36.3701  Train Acc: 55.68%  Val Acc: 72.73%
[raw] Epoch 2/20  Loss: 36.1930  Train Acc: 88.07%  Val Acc: 77.27%
[raw] Epoch 3/20  Loss: 36.0451  Train Acc: 93.18%  Val Acc: 77.27%
[raw] Epoch 4/20  Loss: 35.8888  Train Acc: 96.02%  Val Acc: 77.27%
[raw] Epoch 5/20  Loss: 35.7672  Train Acc: 96.59%  Val Acc: 77.27%
[raw] Epoch 6/20  Loss: 35.7016  Train Acc: 96.59%  Val Acc: 77.27%
[raw] Epoch 7/20  Loss: 35.6142  Train Acc: 96.02%  Val Acc: 79.55%
[raw] Epoch 8/20  Loss: 35.6093  Train Acc: 97.16%  Val Acc: 79.55%
[raw] Epoch 9/20  Loss: 35.5566  Train Acc: 96.59%  Val Acc: 81.82%
[raw] Epoch 10/20  Loss: 35.5101  Train Acc: 97.16%  Val Acc: 81.82%
[raw] Epoch 11/20  Loss: 35.5619  Train Acc: 96.02%  Val Acc: 81.82%
[raw] Epoch 12/20  Loss: 35.5043  Train Acc: 97.73%  Val Acc: 81.82%
[raw] Epoch 13/20  Loss: 35.5125  Train Acc: 96.02%  Val Acc: 81.82%
[raw] Epoch 14/20  Loss: 35.4981  Train Acc: 97.16%  Val Acc: 81.82%
[raw] Epoch 15/20  

In [22]:
evaluate_model(model_raw, raw_test_loader, device, label="raw")
evaluate_model(model_aug, aug_test_loader, device, label="augmented")

[raw] 測試集準確率：77.50%
[augmented] 測試集準確率：85.00%


85.0